In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))
# import pyspark to work on spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('yelp_data_exploration').getOrCreate()
# Any results you write to the current directory are saved as output.

In [5]:
#import yelp review data into dataframe
yelp_review = spark.read.json('data/yelp_academic_dataset_review.json')
# import yelp business data into dataframe
yelp_business = spark.read.json('data/yelp_academic_dataset_business.json')
# import yelp user data into dataframe
yelp_user = spark.read.json('data/yelp_academic_dataset_user.json')
# import yelp tip data into dataframe
#yelp_tip = spark.read.json('../input/yelp_academic_dataset_tip.json')
# import yelp checkin data into dataframe
#yelp_checkin = spark.read.json('../input/yelp_academic_dataset_checkin.json')

**Top 10 Reviewed Business**

In [6]:
# now see top most reviewed business.
# so take review data which has rating(stars) more than 3
review_star_three = yelp_review.filter('stars >3')
grouped_review = review_star_three.groupby('business_id').count()
review_sort = grouped_review.sort('count',ascending=False)

In [7]:
business_only = yelp_business.select('business_id','name','categories')
review_business_name = business_only.join(review_sort,'business_id','inner')
Top_ten_reviewed_business = review_business_name.limit(10)
Top_ten_reviewed_business.show()

+--------------------+--------------------+--------------------+-----+
|         business_id|                name|          categories|count|
+--------------------+--------------------+--------------------+-----+
|VHsNB3pdGVcRgs6C3...|     Dublin O'Neil's|  Restaurants, Irish|   75|
|r-W1HPIZ3V6S2DaXh...|       Allen Hui, OD|Optometrists, Hea...|   59|
|cKwg6HFaLYXl7Ar0r...|AAA Action Garage...|Garage Door Servi...|   12|
|V_maCS_uBRMjqa_BC...| Aroma Waxing Clinic|Day Spas, Beauty ...|   51|
|RMjCnixEY5i12Ciqn...|     Four Star Sushi|Restaurants, Sush...|   45|
|_Wyo52ijeq3QgcmkI...|Lemongrass Restau...|Restaurants, Thai...|   17|
|x1fx7C_tc064cFATj...|The Lodge at Gran...|Restaurants, Bars...|   55|
|ATeOjZzE3VOI-RW7F...|        One Red Door|Cocktail Bars, Br...|   93|
|aiAYA0SKz-wRDj04g...|    Proficient Clean|Home Services, Pr...|   98|
|pK83jG-vw6UykrHkw...|    Poetry Jazz Cafe|Music Venues, Nig...|   38|
+--------------------+--------------------+--------------------+-----+



**Top 10 category which has most business count**

In [8]:
from pyspark.sql.functions import split,explode
category = yelp_business.select('categories')
individual_category = category.select(explode(split('categories', ',')).alias('category'))
grouped_category = individual_category.groupby('category').count()
top_category = grouped_category.sort('count',ascending=False)
top_category.show(10,truncate=False)

+-----------------+-----+
|category         |count|
+-----------------+-----+
| Restaurants     |44488|
| Shopping        |26128|
| Food            |24452|
|Restaurants      |19456|
| Home Services   |16205|
| Beauty & Spas   |14216|
| Health & Medical|13978|
| Local Services  |11958|
| Nightlife       |11415|
| Bars            |10201|
+-----------------+-----+
only showing top 10 rows



**Top Rating give by User to business**

In [9]:
rating = yelp_business.select('stars')
group_rating = rating.groupby('stars').count()
rating_top = group_rating.sort('count',ascending=False)
rating_top.show(truncate=False)

+-----+-----+
|stars|count|
+-----+-----+
|4.0  |39199|
|3.5  |38079|
|4.5  |29940|
|3.0  |28634|
|5.0  |27080|
|2.5  |21435|
|2.0  |13124|
|1.5  |6004 |
|1.0  |5898 |
+-----+-----+



**Top Locations who have number of business more in world**

In [10]:
locations = yelp_business.select('business_id','city')
review_city = yelp_review.select('business_id')
merge_city = locations.join(review_city,'business_id','inner')
grouped_review_city = merge_city.groupby('city').count()
most_reviewed_city = grouped_review_city.groupby('city').sum()
most_reviewed_city.sort('sum(count)',ascending=False).show(10)

+----------+----------+
|      city|sum(count)|
+----------+----------+
| Las Vegas|   2444801|
|   Phoenix|    876619|
|   Toronto|    600584|
|Scottsdale|    457563|
| Charlotte|    384775|
| Henderson|    273735|
|Pittsburgh|    270862|
|     Tempe|    238195|
|      Mesa|    205643|
|  Chandler|    191648|
+----------+----------+
only showing top 10 rows

